In [9]:
#Read txt files into lists for train and test data
import glob
import os
from bs4 import BeautifulSoup 

def cleanMe(html):
    
    soup = BeautifulSoup(html, "lxml") # create a new bs4 object from the html data loaded
    for script in soup(["script", "style"]): # remove all javascript and stylesheet code
        script.extract()
    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

neg_list = sorted(glob.glob(os.path.join(os.getcwd(), "train_neg", "*.txt")))
pos_list = sorted(glob.glob(os.path.join(os.getcwd(), "train_pos", "*.txt")))
neg_testlist = sorted(glob.glob(os.path.join(os.getcwd(), "test_neg", "*.txt")))
pos_testlist = sorted(glob.glob(os.path.join(os.getcwd(), "test_pos", "*.txt")))

neg_train = []
pos_train = []
neg_test = []
pos_test = []

for file_path in neg_list:
    with open(file_path) as f_input:
        neg_train.append(cleanMe(f_input.read()))

for file_path in pos_list:
    with open(file_path) as f_input:
        pos_train.append(cleanMe(f_input.read()))
        
for file_path in neg_testlist:
    with open(file_path) as f_input:
        neg_test.append(cleanMe(f_input.read()))

for file_path in pos_testlist:
    with open(file_path) as f_input:
        pos_test.append(cleanMe(f_input.read()))

train_list = neg_train + pos_train
test_list = neg_test + pos_test


In [10]:
print(len(neg_train))
print(len(pos_train))
print(neg_list[0])
print(neg_train[0])

12500
12500
/Users/huyucong/Desktop/hw1/train_neg/0_3.txt
Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.


In [11]:
# Split train data into actual train and validation sets

train_split = 10000
train_data = neg_train[:train_split] + pos_train[:train_split]
train_targets = [0] * len(neg_train[:train_split]) + [1] * len(pos_train[:train_split])

val_data = neg_train[train_split:] + pos_train[train_split:]
val_targets = [0] * len(neg_train[train_split:]) + [1] * len(pos_train[train_split:])

test_data = test_list
test_targets = [0] * len(neg_test) + [1] * len(pos_test)


print ("Train dataset size is {}".format(len(train_data)))
print ("Val dataset size is {}".format(len(val_data)))
print ("Test dataset size is {}".format(len(test_data)))


Train dataset size is 20000
Val dataset size is 5000
Test dataset size is 25000


In [12]:
print(train_data[0])
print(train_targets[9999])

Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.
0


In [13]:
# Random sample from train dataset
import random
print (train_data[random.randint(0, len(train_data) - 1)])


Star Trek Hidden Frontier will surprise you in many ways. First, it's a fan made series, available only on the web, and it features mainly friends & neighbors who have the computer programs and home video cameras and sewing machines to, as Mickey & Judy once put it, put on a show. It's definitely friends & neighbors to, you can tell. A lot of these people aren't the most beautiful looking folks you've ever seen, or the youngest, or the thinnest
some of them stumble through their lines like they're walking on marbles
some of them have thick accents, or simply don't seem to speak well in the first place, whick makes it virtually impossible to understand a single solitary word that they're saying. Still, you have to admit, for everything these friends & neighbors have put together, it's actually fun to watch. Yes, some of the dialogue is hokey. Yes, it's a little odd (though admittedly a little cool too) watching two Starfleet males kiss (although some of the kissing scenes seem to go on 

Before training the classifier, first we are going to tokenize the dataset using spacy.io

Run (shown in the cell below):

* ```pip install spacy```
* ```python -m spacy download en_core_web_sm```

In [14]:
!pip install spacy
!python -m spacy download en_core_web_sm

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /Users/huyucong/anaconda3/lib/python3.6/site-packages/en_core_web_sm -->
    /Users/huyucong/anaconda3/lib/python3.6/site-packages/spacy/data/en_core_web_sm

    You can now load the model via spacy.load('en_core_web_sm')



In [15]:
# Let's write the tokenization function 

import spacy
import string

# Load English tokenizer, tagger, parser, NER and word vectors
tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation

# lowercase and remove punctuation
def tokenize(sent):
    tokens = tokenizer(sent)
    output = []
    for index in range(0,len(tokens)-1):
        if tokens[index].text not in punctuations and tokens[index+1].text not in punctuations:
            output.append(tokens[index].text+' '+tokens[index+1].text)
    return output

# Example
tokens = tokenize(u"<!DOCTYPE HTML>\n<head>My yardstick for measuring a movie's watch-ability is if I get squirmy. If I start shifting positions and noticing my butt is sore, the film is too long. This movie did not even come close to being boring. Predictable in some parts sure, but never boring.<br /><br />All of the other military branches have had love notes written about them and seen their recruitment levels go up, why not the Coast Guard too? They are definitely under-appreciated, until the day your boat sinks that is.<br /><br />The movie was very enjoyable and fun. Kevin Costner is perfect as the aging macho man who doesn't know when to quit. However, I was most impressed by Ashton Kutcher's performance. ")
print (tokens)

['DOCTYPE HTML', 'My yardstick', 'yardstick for', 'for measuring', 'measuring a', 'a movie', "movie 's", "'s watch", 'ability is', 'is if', 'if I', 'I get', 'get squirmy', 'If I', 'I start', 'start shifting', 'shifting positions', 'positions and', 'and noticing', 'noticing my', 'my butt', 'butt is', 'is sore', 'the film', 'film is', 'is too', 'too long', 'This movie', 'movie did', 'did not', 'not even', 'even come', 'come close', 'close to', 'to being', 'being boring', 'Predictable in', 'in some', 'some parts', 'parts sure', 'but never', 'never boring.<br', 'boring.<br /><br', '/><br />All', '/>All of', 'of the', 'the other', 'other military', 'military branches', 'branches have', 'have had', 'had love', 'love notes', 'notes written', 'written about', 'about them', 'them and', 'and seen', 'seen their', 'their recruitment', 'recruitment levels', 'levels go', 'go up', 'why not', 'not the', 'the Coast', 'Coast Guard', 'Guard too', 'They are', 'are definitely', 'definitely under', 'until t

In [16]:
# This is the code cell that tokenizes train/val/test datasets
# However it takes about 15-20 minutes to run it
# For convinience we have provided the preprocessed datasets
# Please see the next code cell
import pickle as pkl

def tokenize_dataset(dataset):
    token_dataset = []
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later
    all_tokens = []
    
    for sample in dataset:
        tokens = tokenize(sample)
        token_dataset.append(tokens)
        all_tokens += tokens

    return token_dataset, all_tokens

# val set tokens
print ("Tokenizing val data")
val_data_tokens, _ = tokenize_dataset(val_data)
pkl.dump(val_data_tokens, open("val_data_tokens.p", "wb"))

# test set tokens
print ("Tokenizing test data")
test_data_tokens, _ = tokenize_dataset(test_data)
pkl.dump(test_data_tokens, open("test_data_tokens.p", "wb"))

# train set tokens
print ("Tokenizing train data")
train_data_tokens, all_train_tokens = tokenize_dataset(train_data)
pkl.dump(train_data_tokens, open("train_data_tokens.p", "wb"))
pkl.dump(all_train_tokens, open("all_train_tokens.p", "wb"))

Tokenizing val data
Tokenizing test data
Tokenizing train data


In [16]:
# First, download datasets from here
# Use your NYU account
#https://drive.google.com/open?id=1eR2LFI5MGliHlaL1S2nsX4ouIO1k_ip2
#https://drive.google.com/open?id=133QCWbiz_Xc7Qm4r6t-fJP1K669xjNlM
#https://drive.google.com/open?id=1SuUIUpJ1iznU707ktkpnEGSwt_XIqOYp
#https://drive.google.com/open?id=1UQsrZ2LVfcxdxxa47344fMs_qvya72KR

import pickle as pkl

# Then, load preprocessed train, val and test datasets
train_data_tokens = pkl.load(open("train_data_tokens.p", "rb"))
all_train_tokens = pkl.load(open("all_train_tokens.p", "rb"))

val_data_tokens = pkl.load(open("val_data_tokens.p", "rb"))
test_data_tokens = pkl.load(open("test_data_tokens.p", "rb"))

# double checking
print ("Train dataset size is {}".format(len(train_data_tokens)))
print ("Val dataset size is {}".format(len(val_data_tokens)))
print ("Test dataset size is {}".format(len(test_data_tokens)))

print ("Total number of tokens in train dataset is {}".format(len(all_train_tokens)))

Train dataset size is 20000
Val dataset size is 5000
Test dataset size is 25000
Total number of tokens in train dataset is 4146552


Now, we are going to create the vocabulary of most common 10,000 tokens in the training set.

In [17]:
from collections import Counter

max_vocab_size = 10000
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1

def build_vocab(all_tokens):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(max_vocab_size))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token

token2id, id2token = build_vocab(all_train_tokens)
token2id

{'of the': 2,
 'in the': 3,
 'is a': 4,
 'this movie': 5,
 'and the': 6,
 'to be': 7,
 'to the': 8,
 'the film': 9,
 "it 's": 10,
 'the movie': 11,
 'this film': 12,
 'in a': 13,
 'on the': 14,
 'of a': 15,
 'for the': 16,
 'with the': 17,
 'one of': 18,
 "do n't": 19,
 'it is': 20,
 'it was': 21,
 'is the': 22,
 'as a': 23,
 'at the': 24,
 "It 's": 25,
 'with a': 26,
 'from the': 27,
 'in this': 28,
 'as the': 29,
 'I was': 30,
 'to see': 31,
 'out of': 32,
 'this is': 33,
 'that the': 34,
 'This is': 35,
 'movie is': 36,
 'was a': 37,
 "does n't": 38,
 'I have': 39,
 'and I': 40,
 'by the': 41,
 'have been': 42,
 'for a': 43,
 "I 'm": 44,
 'of this': 45,
 "did n't": 46,
 'film is': 47,
 'and a': 48,
 'is not': 49,
 'the first': 50,
 'the story': 51,
 "'s a": 52,
 'if you': 53,
 'all the': 54,
 'the same': 55,
 'the end': 56,
 'a good': 57,
 "ca n't": 58,
 'a lot': 59,
 'but it': 60,
 'that it': 61,
 'the most': 62,
 'I do': 63,
 'and it': 64,
 'of his': 65,
 'to make': 66,
 'It is': 

In [19]:
# Lets check the dictionary by loading random token from it

random_token_id = random.randint(0, len(id2token)-1)
random_token = id2token[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, id2token[random_token_id]))
print ("Token {}; token id {}".format(random_token, token2id[random_token]))

Token id 8043 ; token a variety
Token a variety; token id 8043


In [20]:
# convert token to id in the dataset
def token2index_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data

train_data_indices = token2index_dataset(train_data_tokens)
val_data_indices = token2index_dataset(val_data_tokens)
test_data_indices = token2index_dataset(test_data_tokens)

# double checking
print ("Train dataset size is {}".format(len(train_data_indices)))
print ("Val dataset size is {}".format(len(val_data_indices)))
print ("Test dataset size is {}".format(len(test_data_indices)))

Train dataset size is 20000
Val dataset size is 5000
Test dataset size is 25000


Now we are going to create PyTorch DataLoader 

In [21]:
MAX_SENTENCE_LENGTH = 200

import numpy as np
import torch
from torch.utils.data import Dataset

class NewsGroupDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    
    def __init__(self, data_list, target_list):
        """
        @param data_list: list of newsgroup tokens 
        @param target_list: list of newsgroup targets 

        """
        self.data_list = data_list
        self.target_list = target_list
        assert (len(self.data_list) == len(self.target_list))

    def __len__(self):
        return len(self.data_list)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        
        token_idx = self.data_list[key][:MAX_SENTENCE_LENGTH]
        label = self.target_list[key]
        return [token_idx, len(token_idx), label]

def newsgroup_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    data_list = []
    label_list = []
    length_list = []
    #print("collate batch: ", batch[0][0])
    #batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[2])
        length_list.append(datum[1])
    # padding
    for datum in batch:
        padded_vec = np.pad(np.array(datum[0]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
        data_list.append(padded_vec)
    return [torch.from_numpy(np.array(data_list)), torch.LongTensor(length_list), torch.LongTensor(label_list)]

# create pytorch dataloader
#train_loader = NewsGroupDataset(train_data_indices, train_targets)
#val_loader = NewsGroupDataset(val_data_indices, val_targets)
#test_loader = NewsGroupDataset(test_data_indices, test_targets)

BATCH_SIZE = 32
train_dataset = NewsGroupDataset(train_data_indices, train_targets)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

val_dataset = NewsGroupDataset(val_data_indices, val_targets)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

test_dataset = NewsGroupDataset(test_data_indices, test_targets)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=False)

for i, (data, lengths, labels) in enumerate(train_loader):
    print (data)
    print (labels)
    break

tensor([[ 170,    1, 6516,  ...,    0,    0,    0],
        [   1,  530,    1,  ..., 7243, 1753,  107],
        [   1,    1,    1,  ..., 6568,  289,    1],
        ...,
        [3992,  122, 1234,  ...,    0,    0,    0],
        [1461,    1, 1406,  ...,  206,  590,    1],
        [  25, 6082,  590,  ...,    0,    0,    0]])
tensor([1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
        1, 1, 0, 0, 1, 1, 1, 0])


Here we will define Bag-of-Words model in PyTorch

In [22]:
# First import torch related libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

class BagOfWords(nn.Module):
    """
    BagOfWords classification model
    """
    def __init__(self, vocab_size, emb_dim):
        """
        @param vocab_size: size of the vocabulary. 
        @param emb_dim: size of the word embedding
        """
        super(BagOfWords, self).__init__()
        # pay attention to padding_idx 
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.linear = nn.Linear(emb_dim,20)
    
    def forward(self, data, length):
        """
        
        @param data: matrix of size (batch_size, max_sentence_length). Each row in data represents a 
            review that is represented using n-gram index. Note that they are padded to have same length.
        @param length: an int tensor of size (batch_size), which represents the non-trivial (excludes padding)
            length of each sentences in the data.
        """
        out = self.embed(data)
        out = torch.sum(out, dim=1)
        out /= length.view(length.size()[0],1).expand_as(out).float()
     
        # return logits
        out = self.linear(out.float())
        return out

emb_dim = 100
model = BagOfWords(len(id2token), emb_dim)

In [26]:
learning_rate = 0.01
num_epochs = 10 # number epoch to train

# Criterion and Optimizer
criterion = torch.nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Function for testing the model
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data, lengths, labels in loader:
        data_batch, length_batch, label_batch = data, lengths, labels
        outputs = F.softmax(model(data_batch, length_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        
        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

for epoch in range(num_epochs):
    for i, (data, lengths, labels) in enumerate(train_loader):
        model.train()
        data_batch, length_batch, label_batch = data, lengths, labels
        optimizer.zero_grad()
        outputs = model(data_batch, length_batch)
        loss = criterion(outputs, label_batch)
        loss.backward()
        optimizer.step()
        # validate every 100 iterations
        if i > 0 and i % 100 == 0:
            # validate
            val_acc = test_model(val_loader, model)
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                       epoch+1, num_epochs, i+1, len(train_loader), val_acc))


Epoch: [1/10], Step: [101/625], Validation Acc: 77.6
Epoch: [1/10], Step: [201/625], Validation Acc: 77.88
Epoch: [1/10], Step: [301/625], Validation Acc: 78.0
Epoch: [1/10], Step: [401/625], Validation Acc: 77.54
Epoch: [1/10], Step: [501/625], Validation Acc: 77.98
Epoch: [1/10], Step: [601/625], Validation Acc: 77.78
Epoch: [2/10], Step: [101/625], Validation Acc: 77.9
Epoch: [2/10], Step: [201/625], Validation Acc: 77.94
Epoch: [2/10], Step: [301/625], Validation Acc: 78.06
Epoch: [2/10], Step: [401/625], Validation Acc: 77.9
Epoch: [2/10], Step: [501/625], Validation Acc: 77.84
Epoch: [2/10], Step: [601/625], Validation Acc: 77.96
Epoch: [3/10], Step: [101/625], Validation Acc: 77.96
Epoch: [3/10], Step: [201/625], Validation Acc: 78.06
Epoch: [3/10], Step: [301/625], Validation Acc: 77.88
Epoch: [3/10], Step: [401/625], Validation Acc: 78.04
Epoch: [3/10], Step: [501/625], Validation Acc: 77.98
Epoch: [3/10], Step: [601/625], Validation Acc: 77.98
Epoch: [4/10], Step: [101/625], 

In [27]:
print ("After training for {} epochs".format(num_epochs))
print ("Val Acc {}".format(test_model(val_loader, model)))
print ("Test Acc {}".format(test_model(test_loader, model)))

After training for 10 epochs
Val Acc 77.74
Test Acc 78.424


## Exercise 1
### Try training the model with larger embedding size and for larger number of epochs
### Also plot the training curves of the model

## Exercise 2:
### Try downloading IMDB Large Movie Review Dataset that is used for Assignment 1 http://ai.stanford.edu/~amaas/data/sentiment/
### and tokenize it

## Exercise 3:
### If you have time, after tokenizing the dataset try training Bag-of-Words model on it and report your initial results
### on validation set.